In [1]:
# 导入工具包
import tensorflow as tf
import warnings
from tensorflow.examples.tutorials.mnist import input_data

# 读入数据
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
warnings.filterwarnings("ignore")

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')

In [3]:
# x为训练图像的占位符、y_为训练图像标签的占位符
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])

# 将单张图片从784维向量重新还原为28x28的矩阵图片
x_image = tf.reshape(x, [-1, 28, 28, 1])


#定义第一层卷积层信道为32
out_channl1 = 32
# 第一层卷积层
W_conv1 = weight_variable([5, 5, 1, out_channl1])
b_conv1 = bias_variable([out_channl1])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)


#定义第二层卷积层信道为64
out_channl2 = 64
# 第二层卷积层
W_conv2 = weight_variable([5, 5, out_channl1, out_channl2])
b_conv2 = bias_variable([out_channl2])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [4]:
#定义全连接层函数

def fcn_layer(inputs,           # 输入数据
               input_dim,        # 输入神经元数量
               output_dim,       # 输出神经元数量
               activation=None):  # 激活函数
    # 以截断正态分布的随机数初始化W
    W = tf.Variable(tf.truncated_normal([input_dim, output_dim], stddev=0.1))

    # 以0初始化b
    b = tf.Variable(tf.constant(0.1, shape=[output_dim]))
    
    # 前置计算
    XWb = tf.matmul(inputs, W) + b
    
    if activation is None:
        outputs = XWb
    else:
        outputs = activation(XWb)
    
    return outputs

In [5]:
# 全连接层，输出为120维的向量
out_num = 1024
input_num = int(h_pool2.shape[1]*h_pool2.shape[2]*h_pool2.shape[3])

h_pool2_flat = tf.reshape(h_pool2, [-1, input_num])
h_fc1 = fcn_layer(h_pool2_flat, input_num, out_num, tf.nn.relu)

# 使用Dropout，keep_prob是一个占位符，训练时为0.5，测试时为1
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# 前置计算
forword = fcn_layer(h_fc1_drop, out_num, 10, None)

In [6]:
#设置训练参数
train_epochs = 100 #训练轮数
batch_size = 550 #单次训练样本数（批次大小）
total_batch = int(mnist.train.num_examples/batch_size) # 一轮训练有多少批次
display_step = 2 #显示粒度
learning_rate = 1e-4 #学习率

pred = tf.nn.softmax(forword)

#定义交叉熵损失函数
loss_function = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits_v2(logits=forword,labels=y_))

#梯度下降优化器
optimizer  = tf.train.AdamOptimizer(learning_rate).minimize(loss_function)

#检查预测类别
correct_prediction = tf.equal(tf.argmax(pred, 1),tf.argmax(y_, 1))

#准确率，讲布尔值转化为浮点数，计算平均值
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess = tf.Session() #声明会话
init = tf.global_variables_initializer() #初始化变量
sess.run(init)


In [7]:
# 开始训练
for epoch in range(train_epochs):
    for batch in range(total_batch):
        xs, ys = mnist.train.next_batch(batch_size)#读取批次数据
        sess.run(optimizer, feed_dict={x:xs,y_:ys,keep_prob:0.5}) #执行批次训练
    #total_batch个批次训练完成后，使用验证数据计算误差与准确率；验证集没有分批

    #打印训练过程中的详细信息
    if (epoch+1) % display_step == 0:
        loss,acc = sess.run([loss_function,accuracy],
                       feed_dict={x:mnist.test.images, y_:mnist.test.labels, keep_prob:1})
        print("Train Epoch:", epoch+1, "loss=", loss," Accuracy=",acc)
        
print("Train Finished!")

Train Epoch: 2 loss= 0.18472402  Accuracy= 0.9477
Train Epoch: 4 loss= 0.11866749  Accuracy= 0.9664
Train Epoch: 6 loss= 0.090273485  Accuracy= 0.9731
Train Epoch: 8 loss= 0.07105127  Accuracy= 0.9779
Train Epoch: 10 loss= 0.06324996  Accuracy= 0.9794
Train Epoch: 12 loss= 0.05504989  Accuracy= 0.9819
Train Epoch: 14 loss= 0.049660265  Accuracy= 0.9832
Train Epoch: 16 loss= 0.044511862  Accuracy= 0.9855
Train Epoch: 18 loss= 0.04107039  Accuracy= 0.9861
Train Epoch: 20 loss= 0.038294934  Accuracy= 0.9874
Train Epoch: 22 loss= 0.037633542  Accuracy= 0.9878
Train Epoch: 24 loss= 0.03547846  Accuracy= 0.9886
Train Epoch: 26 loss= 0.03484436  Accuracy= 0.9883
Train Epoch: 28 loss= 0.032332353  Accuracy= 0.989
Train Epoch: 30 loss= 0.030623084  Accuracy= 0.9893
Train Epoch: 32 loss= 0.030709991  Accuracy= 0.9894
Train Epoch: 34 loss= 0.02951186  Accuracy= 0.9905
Train Epoch: 36 loss= 0.030678894  Accuracy= 0.9906
Train Epoch: 38 loss= 0.029725788  Accuracy= 0.9903
Train Epoch: 40 loss= 0.02

In [8]:
print("test accuracy %f" % sess.run(accuracy,feed_dict={
        x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

test accuracy 0.992200


In [9]:
print("validation accuracy %f" % sess.run(accuracy,feed_dict={
        x: mnist.validation.images, y_: mnist.validation.labels, keep_prob: 1.0}))

validation accuracy 0.991800
